In [1]:
import psycopg2
import pandas as pd

In [2]:
# подтягивание аккаунта из excel

profile = pd.read_excel("account.profile.xlsx").astype(str)
profile['profile_tg_id'].astype(str)
profile['profile_role_name'].astype(str)
profile['profile_name'].astype(str)


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in profile.iterrows():
    try:
        cur.execute("""
            INSERT INTO account.profile (profile_tg_id, profile_role_name, profile_name)
            VALUES (%s, %s, %s)
            """,
            (
                row['profile_tg_id'],
                row['profile_role_name'],
                row['profile_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

profile

,profile_tg_id,profile_role_name,profile_name
0,123456789,Сотрудник,Авсиевич Владимирв Викторович
1,987654321,Сотрудник,Болвашенков Иван Викторович
2,234567890,Сотрудник,Буканова Юлия Владимировна
3,345678901,Сотрудник,Гуленко Ольга Николаевна
4,456789012,Сотрудник,Гущин Андрей Викторович
5,567890123,Сотрудник,Дрождин Владимир Викторович
6,678901234,Сотрудник,Кузьмин Андрей Викторович
7,789012345,Сотрудник,Мостовой Илья Леонидович
8,890123456,Сотрудник,Панюкова Екатерина Владимировна
9,901234567,Сотрудник,Серяпин Константин Михайлович


In [3]:
# подтягиваем информацию о расписании из excel

final_res = pd.DataFrame()
for i in range(4):
    exec(f'lessons = pd.read_excel("1 курс_группы_11ТМ_12ИИ_13ИИ_14БТ.xlsx", sheet_name = {i})')
    x = 0 
    for j in range(len(lessons.columns)//13):  # type: ignore
        exec(f'df_{j} = lessons[lessons.columns[{x}:{x+13}].tolist()]')
        exec(f'df_{j}.columns = lessons.columns[:13].tolist()')
        x+=14
    print(i)
    df = pd.concat([df_0,df_1,df_2,df_3,df_4]) # type: ignore
    df = df.dropna(how='all')
    df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')

    result = pd.DataFrame()

    for c in range(1,len(df.columns),3):
        df_copy = df[[df.columns[0],df.columns[c],df.columns[c+1],df.columns[c+2]]]
        df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
        df_date_sub_copy = df_copy.loc[(df_copy[df.columns[c+2]].isna())][[df.columns[0],df.columns[c],df.columns[c+1]]]
        df_copy= df_copy.dropna()
        exec(f'res_{c} = df_copy.merge(df_date_sub_copy, on =[df.columns[0],df.columns[{c}+1]])')
        exec(f'res_{c}["group"] = df.columns[{c}]')
        exec(f'res_{c}.columns = ["Дата занятия","Тип занятия","Время занятия","Аудитория","Предмет","Группа"]')
        exec(f'result = pd.concat([result, res_{c}]).reset_index(drop=True)')
        exec(f'result.dropna(inplace=True)')
    final_res = pd.concat([final_res,result]).reset_index(drop=True)    
final_res['Дата занятия']=final_res['Дата занятия'].str[:-3] + ".2024"


final_res['Время начала занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[0].str.replace('-', ':')
final_res['Время конца занятия'] = final_res['Время занятия'].str.split('\n', expand=True)[1].str.replace('-', ':')
final_res['ссылка'] = 'null'
final_res['Дата занятия'] = pd.to_datetime(final_res['Дата занятия'], format='%d.%m.%Y')
final_res['Время начала занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время начала занятия'].astype(str)
final_res['Время конца занятия'] = final_res['Дата занятия'].astype(str) + ' ' + final_res['Время конца занятия'].astype(str)
final_res['id_учителя']='0'

final_res = final_res[['Группа', 'Тип занятия', 'Аудитория', 'ссылка', 'Время начала занятия', 'Время конца занятия', 'id_учителя', 'Предмет']]

final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лекции'], 'Лекция')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Практические занятия'], 'Практическое занятие')
final_res['Тип занятия'] = final_res['Тип занятия'].replace(['Лабораторные занятия'], 'Лабораторное занятие')

# задаем к каждому предмету id препода

final_res.loc[final_res['Предмет']== 'Технологии AR/VR',                                                    ['id_учителя'] ]= '123456789'
final_res.loc[final_res['Предмет']== '3D анатомия - физиология',                                            ['id_учителя'] ]= '987654321'
final_res.loc[final_res['Предмет']== 'Ознакомительная практика',                                            ['id_учителя'] ]= '234567890'
final_res.loc[final_res['Предмет']== 'Медицинская статистика',                                              ['id_учителя'] ]= '345678901'
final_res.loc[final_res['Предмет']== 'Анализ данных на  Python',                                            ['id_учителя'] ]= '456789012'
final_res.loc[final_res['Предмет']== 'Медицинские аппаратно-программные комплексы',                         ['id_учителя'] ]= '567890123'
final_res.loc[final_res['Предмет']== 'Системный анализ и организация здравоохранения',                      ['id_учителя'] ]= '678901234'
final_res.loc[final_res['Предмет']== 'Методы глубокого обучения',                                           ['id_учителя'] ]= '789012345'
final_res.loc[final_res['Предмет']== 'Системы поддержки принятия решений',                                  ['id_учителя'] ]= '890123456'
final_res.loc[final_res['Предмет']== 'Иностранный язык в профессиональной сфере',                           ['id_учителя'] ]= '901234567'
final_res.loc[final_res['Предмет']== 'Основы молекулярной биологии',                                        ['id_учителя'] ]= '112233445'
final_res.loc[final_res['Предмет']== 'Педагогическая практика',                                             ['id_учителя'] ]= '223344556'
final_res.loc[final_res['Предмет']== 'Базовые аспекты биохимии человека',                                   ['id_учителя'] ]= '334455667'
final_res.loc[final_res['Предмет']== 'Методическое сопровождение применения диагностических тест-систем',   ['id_учителя'] ]= '445566778'
final_res.loc[final_res['Предмет']== 'Биоинформатика',                                                      ['id_учителя'] ]= '556677889'
final_res.loc[final_res['Предмет']== 'Основы медицинской микробиологии и иммунологии',                      ['id_учителя'] ]= '667788990'
final_res.loc[final_res['Предмет']== 'Технологическая акселерация',                                         ['id_учителя'] ]= '778899001'


# загрузка в бд schedule.lesson

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in final_res.iterrows():
    try:
        cur.execute("""
            INSERT INTO schedule.lesson (lesson_type_group_name, lesson_type_education_name, lesson_type_education_room_name,
            lesson_type_education_room_link, lesson_date_time_start, lesson_date_time_end,
            lesson_link, lesson_teacher_id, lesson_subject_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['Группа'],
                row['Тип занятия'],
                row['Аудитория'],
                row['ссылка'],
                row['Время начала занятия'],
                row['Время конца занятия'],
                row['ссылка'],
                row['id_учителя'],
                row['Предмет']
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

final_res

0
1


C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Lo

2
3


C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[df.columns[0]] = df[df.columns[0]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Local\Temp\ipykernel_33444\1728095605.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[df.columns[c+1]] = df_copy[df.columns[c+1]].fillna(method='ffill')
C:\Users\pizdn\AppData\Lo

,Группа,Тип занятия,Аудитория,ссылка,Время начала занятия,Время конца занятия,id_учителя,Предмет
0,11-ТМ,Лекция,ЭИОС,null,2024-09-02 14:40,2024-09-02 15:25,123456789,Технологии AR/VR
1,11-ТМ,Лекция,ЭИОС,null,2024-09-02 15:30,2024-09-02 16:15,123456789,Технологии AR/VR
2,11-ТМ,Лекция,ЭИОС,null,2024-09-02 16:20,2024-09-02 17:05,987654321,3D анатомия - физиология
3,11-ТМ,Лекция,ЭИОС,null,2024-09-02 17:10,2024-09-02 17:55,987654321,3D анатомия - физиология
4,11-ТМ,Практическое занятие,"ауд. УК.2.212 (УК Гагарина, д. 18)",null,2024-09-02 18:00,2024-09-02 18:45,987654321,3D анатомия - физиология
...,...,...,...,...,...,...,...,...
2224,14-БТ,Лекция,ЭИОС,null,2024-11-29 17:10,2024-11-29 17:55,556677889,Биоинформатика
2225,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 18:00,2024-11-29 18:45,223344556,Педагогическая практика
2226,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 18:50,2024-11-29 19:35,223344556,Педагогическая практика
2227,14-БТ,Практическое занятие,"ауд. УК.2.213 (УК Гагарина, д. 18)",null,2024-11-29 19:40,2024-11-29 20:25,223344556,Педагогическая практика


In [4]:
final_res['Предмет'].unique()

array(['Технологии AR/VR', '3D анатомия - физиология',
       'Ознакомительная практика', 'Медицинская статистика',
       'Анализ данных на  Python',
       'Медицинские аппаратно-программные комплексы',
       'Системный анализ и организация здравоохранения',
       'Методы глубокого обучения', 'Системы поддержки принятия решений',
       'Иностранный язык в профессиональной сфере',
       'Основы молекулярной биологии', 'Педагогическая практика',
       'Базовые аспекты биохимии человека',
       'Методическое сопровождение применения диагностических тест-систем',
       'Биоинформатика', 'Основы медицинской микробиологии и иммунологии',
       'Технологическая акселерация'], dtype=object)

In [5]:
# получение кафедр

type_department = pd.read_excel("teacher_info.type_department.xlsx").astype(str)
type_department['кафедра'].astype(str)


# загрезка кафедр в бд teacher_info.type_department

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in type_department.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.type_department (type_department_name)
            VALUES (%s)
            """,
            (
                row['кафедра'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

type_department

,кафедра
0,Кафедра математики
1,Кафедра физики
2,Кафедра информатики
3,Кафедра химии
4,Кафедра биологии
5,Кафедра истории
6,Кафедра экономики
7,Кафедра психологии
8,Кафедра социологии
9,Кафедра философии


In [6]:

teacher = pd.read_excel("teacher_info.teacher.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.teacher (teacher_profile_tg_id, teacher_name, teacher_job, teacher_department, teacher_adress, teacher_email)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (
                row['тг айди'],
                row['фио препода'],
                row['должность'],
                row['кафедра'],
                row['адрес'],
                row['рабочая почта'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher

,тг айди,фио препода,должность,кафедра,адрес,рабочая почта
0,123456789,Авсиевич Владимирв Викторович,"доцент ПИШ, к.т.н.",Кафедра математики,"ул. Ленина, 1",v.v.avsievich@samsmu.ru
1,987654321,Болвашенков Иван Викторович,ассистент ПИШ,Кафедра физики,"пр. Мира, 10",i.v.bolvashenkov@samsmu.ru
2,234567890,Буканова Юлия Владимировна,руководитель образовательной программы ПИШ,Кафедра информатики,"ул. Пушкина, 5",yu.v.bukanova@samsmu.ru
3,345678901,Гуленко Ольга Николаевна,"доцент ПИШ, к.б.н., доцент",Кафедра химии,"ул. Чехова, 12",o.n.gulenko@samsmu.ru
4,456789012,Гущин Андрей Викторович,"доцент ПИШ, к.т.н.",Кафедра биологии,"ул. Горького, 3",a.v.guschin@samsmu.ru
5,567890123,Дрождин Владимир Викторович,"доцент ПИШ, к.т.н., доцент",Кафедра истории,"ул. Советская, 7",v.v.drozhdin@samsmu.ru
6,678901234,Кузьмин Андрей Викторович,"профессор ПИШ, д.т.н., доцент",Кафедра экономики,"ул. Тимирязева, 15",an.v.kuzmin@samsmu.ru
7,789012345,Мостовой Илья Леонидович,ассистент ПИШ,Кафедра психологии,"ул. Лермонтова, 20",i.l.mostovoy@samsmu.ru
8,890123456,Панюкова Екатерина Владимировна,"доцент ПИШ, к.т.н., доцент",Кафедра социологии,"ул. Крылатая, 8",e.v.panyukova@samsmu.ru
9,901234567,Серяпин Константин Михайлович,ассистент ПИШ,Кафедра философии,"ул. Садовая, 4",k.m.seryapin@samsmu.ru


In [7]:

teacher_img = pd.read_excel("teacher_info.image.xlsx")


# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in teacher_img.iterrows():
    try:
        cur.execute("""
            INSERT INTO teacher_info.image (teacher_id, name_img)
            VALUES (%s, %s)
            """,
            (
                row['teacher_id'],
                row['name_img'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

teacher_img

,teacher_id,name_img
0,123456789,img1
1,987654321,img2
2,234567890,img3
3,345678901,img4
4,456789012,img5
5,567890123,img6
6,678901234,img7
7,789012345,img8
8,890123456,img9
9,901234567,img10


In [8]:

adress_contacts_type_place = pd.read_excel("adress_contacts.type_place.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_type_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.type_place (type_place_name)
            VALUES (%s)
            """,
            (
                row['type_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_type_place

,type_place_name
0,Административный корпус
1,Учебный корпус
2,Общежитие
3,Кафедры


In [9]:
adress_contacts_place = pd.read_excel("adress_contacts.place.xlsx").astype(str)
adress_contacts_place['place_latitude'] = adress_contacts_place['place_latitude'].astype(float)
adress_contacts_place['place_longitude'] = adress_contacts_place['place_longitude'].astype(float)

# загрезка кафедр в бд teacher_info.teacher


conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place (place_name, place_time_start, place_time_end, place_phone, place_email, place_adress, place_latitude, place_longitude)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['place_name'],
                row['place_time_start'],
                row['place_time_end'],
                row['place_phone'],
                row['place_email'],
                row['place_adress'],
                row['place_latitude'],
                row['place_longitude'],
            )
        )
        
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place

,place_name,place_time_start,place_time_end,place_phone,place_email,place_adress,place_latitude,place_longitude
0,"Кафе ""Уют""",08:00:00,22:00:00,84951234567,uyut.cafe@example.com,"ул. Счастливая, 10",53.339688,-6.236688
1,"Ресторан ""Вкусно""",12:00:00,23:00:00,84952345678,vkusno.restaurant@example.com,"пр. Ленина, 5",51.339688,-6.236688
2,"Библиотека ""Знание""",09:00:00,20:00:00,84953456789,znanie.library@example.com,"ул. Умная, 15",52.339688,-6.236688
3,"Спортзал ""Фитнес""",06:00:00,22:00:00,84954567890,fitness.gym@example.com,"ул. Спортивная, 20",54.339688,-6.236688
4,"Магазин ""Все для дома""",10:00:00,21:00:00,84955678901,home.store@example.com,"ул. Комфортная, 30",55.339688,-6.236688
5,"Кинотеатр ""Свет""",11:00:00,01:00:00,84956789012,svet.cinema@example.com,"пр. Кино, 8",56.339688,-6.236688
6,"Парк ""Зеленый""",07:00:00,22:00:00,84957890123,green.park@example.com,"ул. Природная, 5",57.339688,-6.236688
7,"Музей ""История""",10:00:00,18:00:00,84958901234,history.museum@example.com,"ул. Культурная, 12",53.339688,-7.236688
8,"Аптека ""Здоровье""",08:00:00,21:00:00,84959012345,zdravie.pharmacy@example.com,"ул. Лекарственная, 7",53.339688,-8.236688
9,"Салон ""Красота""",09:00:00,20:00:00,84950123456,beauty.salon@example.com,"ул. Эстетическая, 3",53.339688,9.236688


In [10]:
adress_contacts_place_info = pd.read_excel("adress_contacts.place_info.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in adress_contacts_place_info.iterrows():
    try:
        cur.execute("""
            INSERT INTO adress_contacts.place_info (place_info_type_place_name, place_info_place_name)
            VALUES (%s, %s)
            """,
            (
                row['place_info_type_place_name'],
                row['place_info_place_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

adress_contacts_place_info

,place_info_type_place_name,place_info_place_name
0,Административный корпус,"Кафе ""Уют"""
1,Учебный корпус,"Ресторан ""Вкусно"""
2,Кафедры,"Библиотека ""Знание"""
3,Учебный корпус,"Спортзал ""Фитнес"""
4,Учебный корпус,"Магазин ""Все для дома"""
5,Общежитие,"Кинотеатр ""Свет"""
6,Общежитие,"Парк ""Зеленый"""
7,Кафедры,"Музей ""История"""
8,Кафедры,"Аптека ""Здоровье"""
9,Учебный корпус,"Салон ""Красота"""


In [11]:
documents_type_group = pd.read_excel("documents.type_group.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_type_group.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.type_group (type_group_name)
            VALUES (%s)
            """,
            (
                row['type_group_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_type_group

,type_group_name
0,Группа 1
1,Группа 2
2,Группа 3


In [12]:
documents_type_file = pd.read_excel("documents.type_file.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_type_file.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.type_file (type_file_name)
            VALUES (%s)
            """,
            (
                row['type_file_name'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_type_file

,type_file_name
0,doc1.docx
1,doc2.docx
2,doc3.docx
3,doc4.docx
4,doc5.docx
5,doc6.pdf
6,doc7.pdf
7,doc8.pdf
8,doc9.pdf


In [13]:
documents_document = pd.read_excel("documents.document.xlsx")

# загрезка кафедр в бд teacher_info.teacher

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="db_schedule",
    user="user",
    password="SamaraSamara"
)

cur = conn.cursor()

for index, row in documents_document.iterrows():
    try:
        cur.execute("""
            INSERT INTO documents.document (file_group, file_name)
            VALUES (%s, %s)
            """,
            (
                row['file_name'],
                row['file_group'],
            )
        )
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

conn.commit()
conn.close()

documents_document

,file_name,file_group
0,Группа 1,doc1.docx
1,Группа 1,doc2.docx
2,Группа 1,doc3.docx
3,Группа 2,doc4.docx
4,Группа 2,doc5.docx
5,Группа 2,doc6.pdf
6,Группа 3,doc7.pdf
7,Группа 3,doc8.pdf
8,Группа 3,doc9.pdf
